In [1]:
import logging
from collections import namedtuple
import random
import math
from numpy.random import choice
import functools

In [2]:
Nimply = namedtuple("Nimply", "row, num_objects")

class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

In [3]:
def _nimsum(state):
    return functools.reduce(lambda a,b : a^b,state)

def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

def optimal(state: Nim) -> Nimply:
    nimsum=_nimsum(state._rows)
    if nimsum==0:
        return pure_random(state)
    else:
        for _ in reversed(range(len(state._rows))):
            if state._rows[_]^nimsum<state._rows[_]:
                return (_,state._rows[_]-(state._rows[_]^nimsum))
        return pure_random(state)

## ***SET OF RULES AND HARD-CODED-AGENT***

In [ ]:
# DATA AND PLYS


def number_active_rows(nim):
    return sum([_>0 for _ in nim.rows])


def indexes_active_rows(nim):
    return [_ for _ in range(len(nim.rows)) if nim.rows[_]>0]

def els_active_rows(nim):
    return [_ for _ in nim.rows if _>0]

def els_max_row(nim):
    return max(els_active_rows(nim))

def avg_els_per_row(nim):
    return sum(nim.rows)/len(nim.rows)

def els_min_row(nim):
    return min(els_active_rows(nim))

def els_second_max_row(nim):
    return sorted(els_active_rows(nim),key=lambda a:a,reverse=True)[1]

def index_max_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a],reverse=True)[0]

def index_min_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[0]

def index_row_middle_els(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[int(len(indexes_active_rows(nim))/2)]

def index_second_max_row(nim):
    return sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a],reverse=True)[1]

def els_row_middle(nim):
    return nim.rows[sorted(indexes_active_rows(nim),key=lambda a:nim.rows[a])[int(len(indexes_active_rows(nim))/2)]]

def sum_rows_odd(nim):
    return sum([True for _ in range(len(nim.rows)) if not nim.rows[_]%2])

def indexes_odd_rows(nim):
    return [_ for _ in range(len(nim.rows)) if not nim.rows[_]%2]

def index_max_odd_row(nim):
    return sorted(indexes_odd_rows(nim),key=lambda a:nim.rows[a],reverse=True)[0]

def index_first_untouched_row(nim,starting_rows):
    for index in range(len(nim.rows)):
        if nim.rows[index]==starting_rows[index]:
            return index

def els_max_odd_row(nim):
    return nim.rows[index_max_odd_row(nim)]

def number_of_rows_with_same_elements_as_starting_elements(nim,starting_rows):
    return sum([nim.rows[_]==starting_rows[_] for _ in range(len(nim.rows))])

def only_odd_rows(nim):
    return all([not nim.rows[_]%2 for _ in indexes_odd_rows(nim)])

# HARD CODED STRAT

def hard_coded_strategy(nim,max_start,starting_rows,rows):
    if number_active_rows(nim)<2:
        return (index_max_row(nim),els_max_row(nim))
    if number_of_rows_with_same_elements_as_starting_elements(nim,starting_rows)>=rows/2:
        return (index_first_untouched_row(nim,starting_rows), 1)
    if number_active_rows(nim)>2:
        return (index_max_row(nim),els_second_max_row(nim))
    if avg_els_per_row(nim)>(max_start/4) and avg_els_per_row(nim)<(max_start/2):
        return (index_max_row(nim),els_max_row(nim)//2)
    if number_active_rows(nim)>rows/2:
        return (index_row_middle_els(nim),els_row_middle(nim))
    if sum_rows_odd(nim):
        return (index_max_odd_row(nim),els_max_odd_row(nim))
    if only_odd_rows(nim) and number_active_rows(nim)>1:
        return (index_max_row(nim),els_second_max_row(nim))

    

    else:
        return pure_random(nim)

## ***EVOLVED AGENT***

In [4]:
# MOVES

def final_move(nim,rows,starting_rows,max_start):
    if number_active_rows(nim)<2:
        return (index_max_row(nim),els_max_row(nim))
    return (0,0)


def same_as_starting_move(nim,rows,starting_rows,max_start):
    if number_of_rows_with_same_elements_as_starting_elements(nim,starting_rows)>=rows/2:
        return (index_first_untouched_row(nim,starting_rows), 1)
    return (0,0)

def pick_max_row_second_max_row(nim,rows,starting_rows,max_start):
    if number_active_rows(nim)>2:
        return (index_max_row(nim),els_second_max_row(nim))
    return (0,0)

def avg_move(nim,rows,starting_rows,max_start):
    if avg_els_per_row(nim)>(max_start/4) and avg_els_per_row(nim)<(max_start/2):
        return (index_max_row(nim),els_max_row(nim)//2)
    return (0,0)

def middle_move(nim,rows,starting_rows,max_start):
    if number_active_rows(nim)>rows/2:
        return (index_row_middle_els(nim),els_row_middle(nim))
    return (0,0)

def odd_move(nim,rows,starting_rows,max_start):
    if sum_rows_odd(nim):
        return (index_max_odd_row(nim),els_max_odd_row(nim))
    return (0,0)
    
def only_odd_move(nim,rows,starting_rows,max_start):
    if only_odd_rows(nim) and number_active_rows(nim)>1:
        return (index_max_row(nim),els_second_max_row(nim))
    return (0,0)


moves=[final_move,same_as_starting_move,pick_max_row_second_max_row,avg_move,middle_move,odd_move,only_odd_move]


def evolved_move(plays,nim,max_start,starting_rows,rows):
    #print(f"Move indexes {}")
    for move in plays:
        ply=moves[move](nim,rows,starting_rows,max_start)
        #print("Ply obtained",ply)
        if ply[1]>0:
            return ply
    return pure_random(nim)

class EvAlgRules:
    def __init__(self,rows,moves):
        self._population_size_=1
        self._offspring_size_=5
        self._games_per_genome_=10
        self._moves=moves
        self._nummoves=len(moves)
        self._number_all_genomes=math.factorial(self._nummoves)
        self._offspring=[]
        self._num_rows_=rows
        self._genomes=[tuple([i for i in range(self._nummoves)]) for _ in range(self._population_size_)]
        self._genomes_visited_already_=set(_ for _ in self._genomes)
        self._population=[(g,self._fitness(g)) for g in self._genomes]
        self._population=sorted(self._population,key=lambda a: a[1],reverse=True)
    
    def _mutate(self,genome):
        points=choice(range(self._nummoves),2,p=[1/self._nummoves for _ in range(self._nummoves)],replace=False)
        return tuple([genome[_] if _ not in points else (genome[points[1]] if _==points[0] else genome[points[0]]) for _ in range(self._nummoves)])


    def get_best_player(self):
        return self._population[0][0]

    def pick_move(self,genome,nim,max_start,starting_rows,rows):
        return evolved_move(genome,nim,max_start,starting_rows,rows)

    def _fitness(self,genome):
        player=0
        starting=random.choice([True,False])
        wins=0
        NUM_GAMES=20
        NUM_ROWS=11
        MAX_START=21
        for _ in range(self._games_per_genome_):
            nim=Nim(11)
            #logging.debug(f"In this game I'm player #{0 if starting else 1}")
            starting_rows=[_ for _ in nim.rows]
            while nim:
                if starting!=player:
                    ply=self.pick_move(genome,nim,MAX_START,starting_rows,NUM_ROWS)
                else:
                    ply=pure_random(nim)
                nim.nimming(ply)
                #logging.debug(f"After player {player} move now rows are {nim}")
                player=1-player
            winner=1-player
            won=(winner==0 and starting) or (winner==1 and not starting)
            if won:
                wins+=1
            starting=random.choice([True,False])
        return wins/self._games_per_genome_


    def __str__(self):
        return f"The best player right now won {self._population[0][1]*self._games_per_genome_} against the pure random bot!"

    def evolve(self,gens):
        print("At beginning the ev was",self)
        for _ in range(gens):
            for __ in range(self._offspring_size_):
                #print(f"Found {len(self._genomes_visited_already_)} states already while max are {self._number_all_genomes}")
                if len(self._genomes_visited_already_)==self._number_all_genomes:
                    print(f"Found all possible combinations already!")
                    print(f"Finished gen {_}, now ev is",self)
                    return
                new_genome=None
                while new_genome is None or new_genome in self._offspring or new_genome in self._population:
                    new_genome=self._mutate(random.choice(self._population)[0])
                self._offspring.append((new_genome,self._fitness(new_genome)))
                self._genomes_visited_already_.add(new_genome)
            self._population=tuple(sorted(list(self._population)+self._offspring,key=lambda a: a[1],reverse=True)[:self._population_size_])
            self._offspring=[]
            print(f"Finished gen {_}, now ev is",self)


myevplayer=EvAlgRules(4,moves)
myevplayer.evolve(10)
print(myevplayer)
best_player=myevplayer.get_best_player()
print(f"Best player is {best_player}")

At beginning the ev was The best player right now won 4.0 against the pure random bot!
Finished gen 0, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 1, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 2, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 3, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 4, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 5, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 6, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 7, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 8, now ev is The best player right now won 9.0 against the pure random bot!
Finished gen 9, now ev is The best player right now won 9.0 against the pure random bot!
The best player right n

### ***GAMES AGAINST THE BEST AGENT FOUND***

In [5]:
wins=0
logging.getLogger().setLevel(logging.DEBUG)
starting=random.choice([True,False])
player=0
NUM_GAMES=20
NUM_ROWS=11
MAX_START=21
for _ in range(NUM_GAMES):
    nim=Nim(NUM_ROWS)
    starting_rows=[_ for _ in nim.rows]
    logging.debug(f"In this game I'm player #{0 if starting else 1}")
    logging.debug(f"Game starting, AM I STARTING??? {starting} with these rows {nim} and this nimsum {_nimsum(nim.rows)}")
    while nim:
        if starting!=player:
            ply=myevplayer.pick_move(best_player,nim,MAX_START,starting_rows,NUM_ROWS)
        else:
            ply=gabriele(nim)
        nim.nimming(ply)
        logging.debug(f"After player {player} play {ply} move now rows are {nim} and nimsum is {_nimsum(nim.rows)}")
        player=1-player
    winner=1-player
    won=(winner==0 and starting) or (winner==1 and not starting)
    if won:
        wins+=1
    starting=random.choice([True,False])

logging.debug(f"After {NUM_GAMES} my player won {wins} games!")

DEBUG:root:In this game I'm player #0
DEBUG:root:Game starting, AM I STARTING??? True with these rows <1 3 5 7 9 11 13 15 17 19 21> and this nimsum 23
DEBUG:root:After player 0 play (0, 1) move now rows are <0 3 5 7 9 11 13 15 17 19 21> and nimsum is 22
DEBUG:root:After player 1 play Nimply(row=1, num_objects=3) move now rows are <0 0 5 7 9 11 13 15 17 19 21> and nimsum is 21
DEBUG:root:After player 0 play (2, 1) move now rows are <0 0 4 7 9 11 13 15 17 19 21> and nimsum is 20
DEBUG:root:After player 1 play Nimply(row=2, num_objects=4) move now rows are <0 0 0 7 9 11 13 15 17 19 21> and nimsum is 16
DEBUG:root:After player 0 play (3, 1) move now rows are <0 0 0 6 9 11 13 15 17 19 21> and nimsum is 17
DEBUG:root:After player 1 play Nimply(row=3, num_objects=6) move now rows are <0 0 0 0 9 11 13 15 17 19 21> and nimsum is 23
DEBUG:root:After player 0 play (4, 1) move now rows are <0 0 0 0 8 11 13 15 17 19 21> and nimsum is 22
DEBUG:root:After player 1 play Nimply(row=4, num_objects=8) mo